In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
from urllib.request import urlparse, urljoin
from more_itertools import one


In [2]:
url= "https://en.tutiempo.net/climate"
page = requests.get(url)
soup =  BeautifulSoup(page.content,'html.parser')

In [3]:
def get_contents(soups):
  results = soups.find(id='ColumnaIzquierda')
#print(results.prettify())
  continent_elems = results.find_all('div', class_='mlistados mt10')
  return continent_elems

def get_urls(elements):
  continents_url = []
  base_url = 'https://en.tutiempo.net'
  for a_tag in elements.find_all('a', href=True):
    urll = urljoin(base_url, a_tag['href'])
    continents_url.append(urll)
  return continents_url

def get_names_list(raw_data):
  raw_cont = raw_data.find_all('a', href=True)
  continents_list = [i.text for i in raw_cont]
  return continents_list

def get_state_urls(raw_data):
  state_urls = []
  for i in raw_data:
    c_urls = get_urls(i)
    state_urls.append(c_urls)
  state_urls = get_flat_list(state_urls)
  return state_urls

def get_state_names_list(raw_data):
  states = []
  for i in raw_data:
    state_names = get_names_list(i)
    states.append(state_names)
  return states

def get_flat_list(list_of_items):
  flat_list = [item for sublist in list_of_items for item in sublist]
  return flat_list

def get_year_urls(soups):
  exceptional_states = soups.find("div", class_="mlistados")
  gdp_table = soups.find("table", attrs={"class": "medias"})

  if gdp_table:
    gdp_table_data = gdp_table.find_all("tr")
    yearly_urls = pop_headings(gdp_table_data)
    flat_list_of_yearly_urls = get_flat_list(yearly_urls)
    return flat_list_of_yearly_urls
  else:
    return get_urls(exceptional_states)

def get_year_names(soups):
  exceptional_states = soups.find("div", class_="mlistados")
  gdp_table = soups.find("table", attrs={"class": "medias"})
  
  if gdp_table:
    gdp_table_data = gdp_table.find_all("tr")
    year_names = get_flat_list(get_state_names_list(gdp_table_data))
    # year_names = list(map(int, year_names))
    return year_names
  else:
    ex_st = get_names_list(exceptional_states)
    ex_state_names = []
    for jj in ex_st:
      result = ''.join([i for i in jj if i.isdigit()])
      ex_state_names.append(str(result))    
    return ex_state_names

def one_element_list(list1):
  if len(list1) == 1:
    return one(list1)
  else:
    return list1[0]

def pop_headings(data):
  year_urls = []
  copy_data = data.copy()
  headings_pop = copy_data.pop(0)
  for i in copy_data:
    year_urls.append(get_urls(i))
  return year_urls

def pop_first_element(list):
  copy_list = list.copy()
  copy_list.pop(0)
  return copy_list

def monthly_avg_data(avg_month_table):
  row1 = avg_month_table.find_all("td")
  rows = []
  if row1:
    for row_data in row1:
      if not None:
        rows.append(row_data.text.replace('\n', '').strip())
      else:
        rows.append(None)
  final_row = pop_first_element(rows)
  return final_row



In [4]:
continent_raw_data = get_contents(soup)
continent_raw_data = one_element_list(continent_raw_data)

#list of continents urls
continent_urls = get_urls(continent_raw_data)

# List of continents
continents_list = get_names_list(continent_raw_data)
print(continents_list)

['Europe', 'Asia', 'Africa', 'North America', 'South America', 'Oceania', 'Antarctica']


# **Europe** **Data**

In [5]:
#south america till colombia done //// and Oceania from Marshall Islands till end done
soup1 = BeautifulSoup((requests.get(continent_urls[6])).content,'html.parser')
country_raw_data = get_contents(soup1)
country_raw_data = one_element_list(country_raw_data)

# List of country urls
country_urls = get_urls(country_raw_data)

#List of Europe countries
country_list = get_names_list(country_raw_data)
print(country_list)
print(len(country_list))

['A028 S. LAW DOME', 'AGO-1', 'AGO-2', 'AGO-3', 'AGO-4', 'AGO-5', 'AGO-6', 'AGO-A81', 'AMERY', 'AMUNDSEN-SCOTT', 'Amundsen-Scott South Pole Station', 'ARCTOWSKI', 'ASUKA JAPAN-BASE']
13


In [6]:
soup2 = BeautifulSoup((requests.get(country_urls[0])).content,'html.parser')
state_raw_data = get_contents(soup2)

#List of state urls
state_urls = get_state_urls(state_raw_data)

#List of states
state_list = get_state_names_list(state_raw_data)
state_list = get_flat_list(state_list)
print(state_list)
print(len(state_list))


['A028 S. LAW DOME', 'AGO-1', 'AGO-2', 'AGO-3', 'AGO-4', 'AGO-5', 'AGO-6', 'AGO-A81', 'AMERY', 'AMUNDSEN-SCOTT', 'Amundsen-Scott South Pole Station', 'ARCTOWSKI', 'ASUKA JAPAN-BASE', 'BAIA TERRA NOVA', 'BASE ARTURO PRAT', 'BASE BELGRANO II', 'BASE ESPERANZA', 'BASE JUBANY', 'BASE MARAMBIO', 'BASE ORCADAS', 'BASE SAN MARTIN', 'BELGRANO 3 ARG-BASE', 'BELLINGSHAUSEN AWS', 'BERNARDO O`HIGGINS', 'BONAPARTE PARKPOINT', 'BROWN ARG-BASE', 'BUTLER ISLAND', 'BYRD', 'BYRD STATION', 'CAPE PHILLIPS', 'CAPE ROSS', 'CASEY', 'CASEY (GC 41 AWS)', 'CASEY (GC 46 AWS)', 'CASEY (GF 08 AWS)', 'CASEY (HALFWAY)', 'CASEY (PETERSON AIR', 'CASEY (POINSETT)', 'CASEY AIRSTRIP AWS', 'CASEY AIRSTRIP(AWS)', 'CASEY BALAENA ISLAN', 'CASEY HAUPT NUNATAK', 'CASEY SNYDER ROCKS', 'CLEAN AIR', 'D-10', 'D-47', 'D-80', 'DAKSHIN GANGOTRI', 'DAVIS', 'DAVIS (LGB 69)', 'DAVIS(LGB 35AWS)', 'DAVIS(LGB 46AWS)', 'DAVIS(LGB 59AWS)', 'DINAMET-URUGUAY', 'DOME C II', 'DOME FUJI', 'DOVERS', 'DRESCHER', 'DRUZHNAYA', 'DUMONT DURVILLE', 'ELA

In [7]:
from datetime import datetime
start_time = datetime.now()


from joblib import Parallel, delayed
import multiprocessing
import time

def dd(ex):
  years_urls = []
  soup3 = BeautifulSoup((requests.get(ex)).content,'html.parser')
  years_urls.append(get_year_urls(soup3))
  return years_urls

def dd1(ex):
  years_list = []
  soup3 = BeautifulSoup((requests.get(ex)).content,'html.parser')
  years_list.append(get_year_names(soup3))
  return years_list

if __name__ == '__main__': 
  pool = multiprocessing.Pool() 
  pool = multiprocessing.Pool(processes=16) 
  results1 = pool.map(dd, state_urls) 
  res = pool.map(dd1, state_urls)
  years_urls = get_flat_list(results1)
  years_list = get_flat_list(res)
  flat_list_years_urls =  get_flat_list(years_urls)
  flat_list_years_list =  get_flat_list(years_list)
  pool.close()
 
  print(len(flat_list_years_urls))
  print(len(flat_list_years_list))
  print(len(years_list))

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

2052
2052
100
Duration: 0:00:12.228699


In [8]:
start_time = datetime.now()

def kk(k):
  month_urls = []
  for j in k:
    soup4 = BeautifulSoup((requests.get(j)).content,'html.parser')
    month_eles = get_contents(soup4)
    month_urls.append(get_urls(one_element_list(month_eles)))
  return month_urls

def kk1(k):
  month_names = []
  for j in k:
    soup4 = BeautifulSoup((requests.get(j)).content,'html.parser')
    month_eles = get_contents(soup4)
    month_names.append(get_names_list(one_element_list(month_eles)))
  return month_names

if __name__ == '__main__': 
  pool = multiprocessing.Pool() 
  pool = multiprocessing.Pool(processes=16) 
  results1 = pool.map(kk, years_urls) 
  res = pool.map(kk1, years_urls)
  month_urls = get_flat_list(results1)
  month_names = get_flat_list(res)
  flat_list_month_names = get_flat_list(month_names)
  flat_list_month_urls = get_flat_list(month_urls)

  print(month_urls)
  print(len(flat_list_month_names))
  print(len(flat_list_month_urls))
 # pool.close()

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))


[['https://en.tutiempo.net/climate/08-2001/ws-898120.html', 'https://en.tutiempo.net/climate/09-2001/ws-898120.html', 'https://en.tutiempo.net/climate/10-2001/ws-898120.html', 'https://en.tutiempo.net/climate/11-2001/ws-898120.html', 'https://en.tutiempo.net/climate/12-2001/ws-898120.html'], ['https://en.tutiempo.net/climate/01-2002/ws-898120.html', 'https://en.tutiempo.net/climate/02-2002/ws-898120.html', 'https://en.tutiempo.net/climate/03-2002/ws-898120.html', 'https://en.tutiempo.net/climate/04-2002/ws-898120.html', 'https://en.tutiempo.net/climate/05-2002/ws-898120.html', 'https://en.tutiempo.net/climate/06-2002/ws-898120.html', 'https://en.tutiempo.net/climate/07-2002/ws-898120.html', 'https://en.tutiempo.net/climate/08-2002/ws-898120.html', 'https://en.tutiempo.net/climate/09-2002/ws-898120.html', 'https://en.tutiempo.net/climate/10-2002/ws-898120.html', 'https://en.tutiempo.net/climate/11-2002/ws-898120.html', 'https://en.tutiempo.net/climate/12-2002/ws-898120.html'], ['https:/

In [9]:
start_time = datetime.now()

def gg(g):
  total_months_per_state_count = []
 # for i in years_urls:
  count = 0
  for j in g:
    soup4 = BeautifulSoup((requests.get(j)).content,'html.parser')
    month_eles = get_contents(soup4)
    months_count_per_year = get_names_list(one_element_list(month_eles))
    count += len(months_count_per_year)
  total_months_per_state_count.append(count)

  return total_months_per_state_count

if __name__ == '__main__': 
  pool = multiprocessing.Pool() 
  pool = multiprocessing.Pool(processes=16) 
  results1 = pool.map(gg, years_urls) 
  c = get_flat_list(results1)

 # flat_list_month_names = get_flat_list(c)
 # flat_list_month_urls = get_flat_list(month_urls)

years = []
len_of_months_in_year = []
for j in month_names:
  len_of_months_in_year.append(len(j))
for i, j in enumerate(flat_list_years_list):
  years.append([j]*len_of_months_in_year[i])

states = []
for i, j in enumerate(state_list):
  states.append([j]*c[i])

flat_list_years = get_flat_list(years)
flat_list_states = get_flat_list(states)

print(len(flat_list_years))
print(len(flat_list_states))
print(flat_list_years)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

20502
20502
['2001', '2001', '2001', '2001', '2001', '2002', '2002', '2002', '2002', '2002', '2002', '2002', '2002', '2002', '2002', '2002', '2002', '2003', '2003', '2003', '2003', '2003', '2003', '2003', '2003', '2003', '2003', '2003', '2003', '2004', '2004', '2004', '2004', '2004', '2004', '2004', '2004', '2004', '2004', '2004', '2004', '2005', '2005', '2005', '2005', '2005', '2005', '2005', '2006', '2008', '2008', '2008', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2011', '2011', '2011', '2011', '2011', '2011', '2011', '2011', '2011', '2011', '2011', '2011', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2008', '2008', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2009', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2011', '2011', '20

In [10]:
head_rows = []
soup = BeautifulSoup((requests.get(flat_list_month_urls[0])).content,'html.parser')
month_data_table = soup.find("table", attrs={"class": "medias mensuales numspan"})
month_headings_raw_data = month_data_table.find_all('tr')
for table_data in month_headings_raw_data:
  row = table_data.find_all("th")
  for row_data in row:
    head_rows.append(row_data.text)
head_rows.pop(0)
head_rows.pop(-1)

'Monthly means and totals:'

In [11]:
start_time = datetime.now()

def Months_datafame(raw_month_urls):
  month_data = []
  for i in raw_month_urls:
    soup = BeautifulSoup((requests.get(i)).content,'html.parser')
    aa = soup.find("table", attrs={"class": "medias mensuales numspan"})
    if aa:
      avg_month_table_raw_data = soup.find("table", attrs={"class": "medias mensuales numspan"}).find_all("tr")[-1]
      month_data.append(monthly_avg_data(avg_month_table_raw_data))
    else: 
      month_data.append([None])     
  return month_data


if __name__ == '__main__': 
  pool = multiprocessing.Pool() 
  pool = multiprocessing.Pool(processes=32) 
  results = pool.map(Months_datafame, month_urls)
  month_df = get_flat_list(results)

  df = pd.DataFrame(month_df, columns=head_rows)
  pool.close()


#df = pd.DataFrame(Months_datafame(flat_list_month_urls), columns=head_rows)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:17:57.408453


In [12]:
state_df = df.copy()
state_df

,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG
0,-36.9,-36,-38,-,-,0,-,-,-,,0,0,0,0
1,-28.4,-27.7,-29.3,-,-,0,-,35.9,61.3,,0,0,0,0
2,-28.8,-28.1,-29.9,-,-,0,-,37.4,62.2,,0,0,0,0
3,-24.3,-23.4,-25.5,-,-,0,-,32.5,52,,0,0,0,0
4,-16.4,-15.8,-17.4,-,-,0,-,27.7,47.9,,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20497,-13.2,-13,-13.4,-,-,0,-,-,-,,0,0,0,0
20498,-13.7,-13.3,-14.3,-,-,0,-,-,-,,0,0,0,0
20499,-17.7,-17.3,-18.1,-,-,0,-,-,-,,0,0,0,0
20500,-20.1,-18.3,-21.8,-,-,0,-,-,-,,0,0,0,0


In [ ]:
#c_df = pd.DataFrame()
#c_df = c_df.append(state_df, ignore_index=True)
#c_df.to_csv("test")
#aa = pd.read_csv("ggg.csv")
#c_df = pd.read_csv("sss.csv")
#c_df = c_df.append(aa, ignore_index=True)
#c_df.to_csv("final.csv")
#c_df


In [ ]:
state_df = c_df.copy()
state_df

In [18]:
state_df["Month"] = flat_list_month_names
state_df["Year"] = flat_list_years
state_df["State"] = flat_list_states
state_df["Country"] = "None"#country_list[6]
state_df

,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG,Month,Year,State,Country
0,-36.9,-36,-38,-,-,0,-,-,-,,0,0,0,0,August,2001,A028 S. LAW DOME,None
1,-28.4,-27.7,-29.3,-,-,0,-,35.9,61.3,,0,0,0,0,September,2001,A028 S. LAW DOME,None
2,-28.8,-28.1,-29.9,-,-,0,-,37.4,62.2,,0,0,0,0,October,2001,A028 S. LAW DOME,None
3,-24.3,-23.4,-25.5,-,-,0,-,32.5,52,,0,0,0,0,November,2001,A028 S. LAW DOME,None
4,-16.4,-15.8,-17.4,-,-,0,-,27.7,47.9,,0,0,0,0,December,2001,A028 S. LAW DOME,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20497,-13.2,-13,-13.4,-,-,0,-,-,-,,0,0,0,0,March,2018,MOUNT SIPLE,None
20498,-13.7,-13.3,-14.3,-,-,0,-,-,-,,0,0,0,0,April,2018,MOUNT SIPLE,None
20499,-17.7,-17.3,-18.1,-,-,0,-,-,-,,0,0,0,0,November,2018,MOUNT SIPLE,None
20500,-20.1,-18.3,-21.8,-,-,0,-,-,-,,0,0,0,0,September,2019,MOUNT SIPLE,None


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
import pandas as pd
#new_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Oceania.csv")
#new_df = pd.DataFrame()
#new_df = new_df.append(state_df, ignore_index=True)
#new_df = new_df.drop(columns=["Unnamed: 0"])
#new_df["Continent"] = "Antarctica"
#new_df.to_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Antarctica.csv")
#new_df.drop(new_df.head(2199).index,inplace=True) # drop last n rows
#new_df

#a = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Asia_and_Europe.csv")
#a = a.drop(columns=["Unnamed: 0"])
#b = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Africa.csv")
#c = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/North_America.csv")
#d = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/South-America.csv")
#e = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Oceania.csv")
#f = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Antarctica.csv")
#f = f.drop(columns=["Unnamed: 0"])

#Final_climate_data_all_continents = pd.concat([a, b, c, d, e, f], ignore_index=True)
Final_climate_data_all_continents.to_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Final_climate_data_all_continents.csv")
#Final_climate_data_all_continents.to_excel("/content/drive/My Drive/Colab Notebooks/Sensegrass/Final_climate_data_all_continents.xlsx")



In [ ]:
final_states_df = new_df.copy()

In [ ]:
#new_df = Months_datafame(flat_list_month_urls)
final_states_df["Month"] = flat_list_month_names
final_states_df["Year"] = flat_list_years
final_states_df["State"] = flat_list_states
final_states_df["Country"] = country_list[12]

In [ ]:
final_states_df.to_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/India.csv")
final_states_df

,Unnamed: 0,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG,Month,Year,State,Country
0,0.0,17.3,25.9,9.7,1013,71.8,0,9.7,4.2,11.3,NaN,0,0,0,6,January,1945,Agartala,India
1,1.0,30.2,33.8,24.9,1007.7,59.4,0,130.5,13.5,22.9,NaN,7,0,3,0,April,1945,Agartala,India
2,2.0,29.1,32.4,24.7,1004,73.1,0,137.8,14.6,24.5,NaN,14,0,8,0,May,1945,Agartala,India
3,3.0,18.1,24.7,11.1,1015.7,67.6,1.02,5.1,4.2,10.3,NaN,1,0,0,0,January,1961,Agartala,India
4,4.0,18.2,23.4,11.7,1011.3,63.9,11.17,5.4,7.3,12.8,NaN,2,0,0,1,February,1961,Agartala,India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27077,NaN,20.5,28.8,12.5,1013.8,34.3,0,3.5,6.1,10.9,,0,0,0,1,February,2020,JAISALMER,India
27078,NaN,23.8,31.2,16.9,1009.2,48.1,37.1,3.5,6.8,12.8,,3,0,2,0,March,2020,JAISALMER,India
27079,NaN,31.5,38.8,24.2,1005.8,33.4,12.44,3.7,6.6,12,,0,0,2,0,April,2020,JAISALMER,India
27080,NaN,35.3,43.3,27.1,1000.4,31.9,1.02,3.3,10.2,18.4,,0,0,6,0,May,2020,JAISALMER,India


In [ ]:
#eu1 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Albania.csv")
#eu2 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Austria.csv")
#eu3 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/Belarus.csv")
#europe = pd.concat([eu1, eu2, eu3], ignore_index=True)
#europe

In [ ]:
#europe = europe.drop(columns=["Unnamed: 0.1"])


In [ ]:
#europe["Continent"] = continents_list[0]
#europe

In [ ]:
#asia = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Sensegrass/India.csv")
#asia["Continent"] = continents_list[1]

In [ ]:
#asia = asia.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])

In [ ]:
#f_df = pd.concat([europe, asia], ignore_index=True)


In [ ]:
#f_df.to_excel("/content/drive/My Drive/Colab Notebooks/Sensegrass/Climate_EU_Asia.xlsx")

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

inputs = range(10) 
def processInput(i):
    return i * i

num_cores = multiprocessing.cpu_count()

results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)
print(results)

In [ ]:
from datetime import datetime
start_time = datetime.now()
# do your work here

month_urls = []
month_names = []

#for i in flat_list_years_urls:  
#  soup4 = BeautifulSoup((requests.get(i)).content,'html.parser')
#  month_eles = get_contents(soup4)
#  month_urls.append(get_urls(one_element_list(month_eles)))
#  month_names.append(get_names_list(one_element_list(month_eles)))

total_months_per_state_count = []

for i in years_urls:
  count = 0
  for j in i:
    soup4 = BeautifulSoup((requests.get(j)).content,'html.parser')
    month_eles = get_contents(soup4)
    month_urls.append(get_urls(one_element_list(month_eles)))
    month_names.append(get_names_list(one_element_list(month_eles)))
    months_count_per_year = get_names_list(one_element_list(month_eles))
    count += len(months_count_per_year)
  total_months_per_state_count.append(count)


years = []
len_of_months_in_year = []
for j in month_names:
  len_of_months_in_year.append(len(j))
for i, j in enumerate(flat_list_years_list):
  years.append([j]*len_of_months_in_year[i])

states = []
for i, j in enumerate(state_list):
  states.append([j]*total_months_per_state_count[i])

#print(states)
#print(len(len_of_months_in_year))
#print(len(flat_list_years_list))
#print(month_urls)

flat_list_month_names = get_flat_list(month_names)
flat_list_month_urls = get_flat_list(month_urls)
flat_list_years = get_flat_list(years)
flat_list_states = get_flat_list(states)

print(len(flat_list_month_names))
print(len(flat_list_month_urls))
print(len(flat_list_years))
print(len(flat_list_states))

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
def Months_datafame(raw_month_urls):
  month_data = []
  for i in raw_month_urls:
    soup = BeautifulSoup((requests.get(i)).content,'html.parser')
    avg_month_table_raw_data = soup.find("table", attrs={"class": "medias mensuales numspan"}).find_all("tr")[-1]
    month_data.append(monthly_avg_data(avg_month_table_raw_data))
  return month_data
 # print(month_data)
 # print(head_rows)

In [ ]:
# from datetime import datetime
# start_time = datetime.now()

# years_urls = []
# years_list = []

# for i in state_urls:
#   soup3 = BeautifulSoup((requests.get(i)).content,'html.parser')
#   #List of year urls
#   years_urls.append(get_year_urls(soup3))

#   #List of year names
#   years_list.append(get_year_names(soup3))

# flat_list_years_urls =  get_flat_list(years_urls)
# flat_list_years_list =  get_flat_list(years_list)
# print(len(years_list))

# end_time = datetime.now()
# print('Duration: {}'.format(end_time - start_time))

In [ ]:
#new start from 25 to 30

# for i in state_urls:
#   soup3 = BeautifulSoup((requests.get(i)).content,'html.parser')
#   #List of year urls
#   years_urls.append(get_year_urls(soup3))

  #List of year names
#   years_list.append(get_year_names(soup3))

# flat_list_years_urls =  get_flat_list(years_urls)
# flat_list_years_list =  get_flat_list(years_list)
# print(len(years_list))
